In [1]:
import pandas as pd
import json
import re
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
import pickle
import ast

from flask import jsonify,make_response
from flask import jsonify
import json 

from flask import Flask
app = Flask(__name__)

In [2]:
with open('vectorizer.pkl','rb') as f:
    vectorizer = pickle.load(f)

In [3]:
with open('RandomForest_classifier_Feb24.pkl','rb') as f:
    RandomForest_model = pickle.load(f)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
def BinningData(TransactionDf, bins):
    min_val = min((TransactionDf['AmountInGBP'])-1)
    max_val = max(TransactionDf['AmountInGBP'])
    #print(min_val, max_val)
    custom_bucket_array = np.linspace(min_val, max_val, bins)
    cut_points = list(custom_bucket_array)
    group_name = ["low","medium","high"]
    TransactionDf["AmountInGBP_bin"] = pd.cut(TransactionDf["AmountInGBP"], cut_points, labels=group_name)
    return TransactionDf
def getMerchfromDesc(tranType,tranDesc):
    if tranType=='POS':        
        parsed=tranDesc.split(', ')
        lenPar=len(parsed)
        if parsed[lenPar-1] == 'REFUND':
            merch=parsed[1:-2]
            return ('').join(merch)+'REFUND'
        else:
            merch=parsed[1:-1]
            return ('').join(merch)
    else:
        return tranDesc
def cleanMerchant(txt):
    txt=re.sub(r'[^a-zA-Z\b]', ' ', str(txt).upper())
    return " ".join(txt.split())


def predictionModule(row):
    if (row['Type']=='ATM'):
        if row['CreditDebitIndicator']=='Debit':
            return row['Type']+' Withdrawal'
        else:
            return row['Type']+' Deposit'
    else:
        vector=vectorizer.transform([row['Predictorcolumn']])
        prediction=RandomForest_model.predict(vector)[0]
        return prediction


In [5]:
@app.route('/getPredictions')
def get_prediction():
    with open('TransformedSummary.json', encoding='ISO-8859-1') as f:
        data = json.load(f)
    TransactionDf = [item for items in data.values() for item in items]
    TransactionDf=pd.DataFrame(TransactionDf)
    TransactionDf['AmountInGBP']=TransactionDf['Amount'].map(lambda x :ast.literal_eval(x)['Amount']).astype(np.float)
    TransactionDf['Type']=TransactionDf['BankTransactionCode'].map(lambda x :ast.literal_eval(x[:-1]+"\"\""+'}')['Code'])
    
    BinningData(TransactionDf, 4)      
    TransactionDf['Merchant']=TransactionDf.apply(lambda row:getMerchfromDesc(row['Type'],row['TransactionInformation']),axis=1 )
    TransactionDf['cleanedMerchant']=TransactionDf['Merchant'].map(lambda x:cleanMerchant(x))

    TransactionDf['Predictorcolumn']=TransactionDf['CreditDebitIndicator']+' '+TransactionDf['cleanedMerchant']+' '+TransactionDf['Type']+' '+TransactionDf['AmountInGBP_bin'].astype(str)
    
    TransactionDf['RF_Predictions']=TransactionDf.apply(lambda row:predictionModule(row),axis=1)
    #surrogate = final_default_list[['New_ID_x','final_predict','Reason']]
    surrogate = TransactionDf[['AccountId','TransactionInformation','Type','RF_Predictions']]
    #final_surrogate = surrogate.drop_duplicates()
    
    dict_lst = []
    for i in surrogate.iterrows():
        #print(i[1]['predict_class'])
        cust_id = i[1]['AccountId']
        Transaction_Information = i[1]['TransactionInformation']
        Transaction_Type = i[1]['Type']
        predicted_class = i[1]['RF_Predictions']
        #default_Reason = i[1]['Reason']

        elm = {"cust_id": cust_id,"Transaction_Information":Transaction_Information,"Transaction_Type":Transaction_Type,"predicted_class":predicted_class}
        dict_lst.append(elm)

        res_data_dict = {"Transaction_Details":dict_lst}
        #print(res_data_dict)
    
        response = app.response_class(response = json.dumps(res_data_dict),status = 200,mimetype='application/json')
        return (response)

In [6]:
if __name__ == '__main__':
    app.run(port=8080,host='0.0.0.0',debug=True)

 * Restarting with stat


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
